### Data Introduction and Outline

The data used is ad impression / click logs (26 million records) of randomly sampled 1,140,000 users from the website of Taobao for 8 days (2017-05-05 16:00:00 to 2017-05-13 15:59:46). Files description is as follows:

* click through rate - click info 'raw_sample.csv'
* user profile -'user_profile.csv'
* advertises info - 'ad_feature.csv'
* log behavior info - 'behavior_log.csv'

Below we:
1. join the file "raw_sample" with "ad_feature" on the key of ad_id 
2. then further join with the file "user_profile" on the key of user_id.
3. save the master table to "ctr_ad_user.csv"
4. extract the first day data to "ctr_ad_user_test.csv" to test function

See "User_Behavior_Exploration.ipynb" and "CTR_EDA.ipynb" for further analysis.

In [53]:
import pandas as pd

#### 1. An overview of CTR info

In [54]:
ctr = pd.read_csv('data/raw_sample.csv')

In [55]:
ctr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26557961 entries, 0 to 26557960
Data columns (total 6 columns):
user          int64
time_stamp    int64
adgroup_id    int64
pid           object
nonclk        int64
clk           int64
dtypes: int64(5), object(1)
memory usage: 1.2+ GB


In [56]:
ctr['time'] = pd.to_datetime(ctr['time_stamp'], unit='s')

In [57]:
ctr.head()

,user,time_stamp,adgroup_id,pid,nonclk,clk,time
0,581738,1494137644,1,430548_1007,1,0,2017-05-07 06:14:04
1,449818,1494638778,3,430548_1007,1,0,2017-05-13 01:26:18
2,914836,1494650879,4,430548_1007,1,0,2017-05-13 04:47:59
3,914836,1494651029,5,430548_1007,1,0,2017-05-13 04:50:29
4,399907,1494302958,8,430548_1007,1,0,2017-05-09 04:09:18


In [58]:
ctr['time'].min(), ctr['time'].max()

(Timestamp('2017-05-05 16:00:00'), Timestamp('2017-05-13 15:59:46'))

The entire dataset is from 2017-05-05 16:00:00 to 2017-05-13 15:59:46. 

#### 2.  Match the ad info with the CTR data

In [59]:
ad = pd.read_csv('data/ad_feature.csv')

In [60]:
ad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846811 entries, 0 to 846810
Data columns (total 6 columns):
adgroup_id     846811 non-null int64
cate_id        846811 non-null int64
campaign_id    846811 non-null int64
customer       846811 non-null int64
brand          600481 non-null float64
price          846811 non-null float64
dtypes: float64(2), int64(4)
memory usage: 38.8 MB


In [61]:
ad.describe()

,adgroup_id,cate_id,campaign_id,customer,brand,price
count,846811.000000,846811.000000,846811.000000,846811.000000,600481.000000,8.468110e+05
mean,423406.000000,5868.593464,206552.604280,113180.406006,229254.422596,1.838867e+03
std,244453.423738,2705.171203,125192.340908,73435.834950,132288.849633,3.108877e+05
min,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000e-02
25%,211703.500000,4383.000000,97583.000000,47960.000000,110616.000000,4.900000e+01
50%,423406.000000,6183.000000,200780.000000,107278.000000,234423.000000,1.390000e+02
75%,635108.500000,7047.000000,314565.500000,172013.500000,344818.000000,3.520000e+02
max,846811.000000,12960.000000,423436.000000,255875.000000,461497.000000,1.000000e+08


In [62]:
ad.head()

,adgroup_id,cate_id,campaign_id,customer,brand,price
0,63133,6406,83237,1,95471.0,170.00
1,313401,6406,83237,1,87331.0,199.00
2,248909,392,83237,1,32233.0,38.00
3,208458,392,83237,1,174374.0,139.00
4,110847,7211,135256,2,145952.0,32.99


In [63]:
ctr_ad = ctr.join(ad.set_index('adgroup_id'), on='adgroup_id')

In [64]:
ctr_ad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26557961 entries, 0 to 26557960
Data columns (total 12 columns):
user           int64
time_stamp     int64
adgroup_id     int64
pid            object
nonclk         int64
clk            int64
time           datetime64[ns]
cate_id        int64
campaign_id    int64
customer       int64
brand          float64
price          float64
dtypes: datetime64[ns](1), float64(2), int64(8), object(1)
memory usage: 2.4+ GB


#### 3.  Match the user info with the CTR data

In [65]:
user = pd.read_csv('data/user_profile.csv')

In [66]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1061768 entries, 0 to 1061767
Data columns (total 9 columns):
userid                   1061768 non-null int64
cms_segid                1061768 non-null int64
cms_group_id             1061768 non-null int64
final_gender_code        1061768 non-null int64
age_level                1061768 non-null int64
pvalue_level             485851 non-null float64
shopping_level           1061768 non-null int64
occupation               1061768 non-null int64
new_user_class_level     716848 non-null float64
dtypes: float64(2), int64(7)
memory usage: 72.9 MB


In [67]:
user.describe()

,userid,cms_segid,cms_group_id,final_gender_code,age_level,pvalue_level,shopping_level,occupation,new_user_class_level
count,1.061768e+06,1.061768e+06,1.061768e+06,1.061768e+06,1.061768e+06,485851.000000,1.061768e+06,1.061768e+06,716848.000000
mean,5.713104e+05,1.512998e+01,5.542412e+00,1.644445e+00,3.410641e+00,1.759850,2.716854e+00,5.623074e-02,2.516379
std,3.295021e+05,2.535503e+01,3.178516e+00,4.786814e-01,1.227458e+00,0.581303,5.861036e-01,2.303669e-01,0.928334
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000,1.000000e+00,0.000000e+00,1.000000
25%,2.862458e+05,0.000000e+00,3.000000e+00,1.000000e+00,3.000000e+00,1.000000,3.000000e+00,0.000000e+00,2.000000
50%,5.714555e+05,0.000000e+00,4.000000e+00,2.000000e+00,3.000000e+00,2.000000,3.000000e+00,0.000000e+00,2.000000
75%,8.546662e+05,2.000000e+01,9.000000e+00,2.000000e+00,4.000000e+00,2.000000,3.000000e+00,0.000000e+00,3.000000
max,1.141729e+06,9.600000e+01,1.200000e+01,2.000000e+00,6.000000e+00,3.000000,3.000000e+00,1.000000e+00,4.000000


In [68]:
user.head()

,userid,cms_segid,cms_group_id,final_gender_code,age_level,pvalue_level,shopping_level,occupation,new_user_class_level
0,234,0,5,2,5,NaN,3,0,3.0
1,523,5,2,2,2,1.0,3,1,2.0
2,612,0,8,1,2,2.0,3,0,NaN
3,1670,0,4,2,4,NaN,1,0,NaN
4,2545,0,10,1,4,NaN,3,0,NaN


In [69]:
ctr_ad_user = ctr_ad.join(user.set_index('userid'), on='user')

In [70]:
ctr_ad_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26557961 entries, 0 to 26557960
Data columns (total 20 columns):
user                     int64
time_stamp               int64
adgroup_id               int64
pid                      object
nonclk                   int64
clk                      int64
time                     datetime64[ns]
cate_id                  int64
campaign_id              int64
customer                 int64
brand                    float64
price                    float64
cms_segid                float64
cms_group_id             float64
final_gender_code        float64
age_level                float64
pvalue_level             float64
shopping_level           float64
occupation               float64
new_user_class_level     float64
dtypes: datetime64[ns](1), float64(10), int64(8), object(1)
memory usage: 4.0+ GB


In [71]:
ctr_ad_user.to_csv(r"data/ctr_ad_user.csv")

#### 4. Extract a sample to test function

We extract a subset of the data to test functions. The subset data is from 2017-05-05 16:00:00 to 2017-05-06 15:59:46.

In [72]:
cut_low = pd.to_datetime('2017-05-05 16:00:00')
cut_up = pd.to_datetime('2017-05-06 15:59:59')

mask_ctr = (ctr['time'] > cut_low) & (ctr['time'] < cut_up)

In [73]:
ctr_test = ctr_ad_user[mask_ctr]

In [74]:
ctr_test.to_csv(r"data/test/ctr_ad_user_test.csv")

In [75]:
!ls -lh data/

total 55698520
-rw-r--r--@ 1 elainewu  staff    30M Jun  7  2017 ad_feature.csv
-rw-r--r--@ 1 elainewu  staff    22G Jun  4  2017 behavior_log.csv
-rw-r--r--  1 elainewu  staff   219M Oct 16 10:54 behavior_test.csv
-rw-r--r--  1 elainewu  staff   3.2G Oct 16 11:42 ctr_ad_user.csv
-rw-r--r--@ 1 elainewu  staff   1.0G Jun  7  2017 raw_sample.csv
drwxr-xr-x  6 elainewu  staff   204B Oct 15 13:50 test
-rw-r--r--@ 1 elainewu  staff    23M Jun  4  2017 user_profile.csv


In [13]:
behavior.head()

,user,time_stamp,btag,cate,brand
0,558157,1493741625,pv,6250,91286
1,558157,1493741626,pv,6250,91286
2,558157,1493741627,pv,6250,91286
3,728690,1493776998,pv,11800,62353
4,332634,1493809895,pv,1101,365477
